# About
This time I decided to follow a completely differnt approach than in the last Tabular Playground Competitions: Use Auto ML Tools.

I will NOT do EDA. 

I will NOT learn anything about the data. 

I will just throw it into an Auto ML Tool and observe the result. In this notebook, it's **Pycaret**.

(Somehow I already feel bad... I love EDA... but hey, I think it's time to try out something new this time)

.................

**Update**: in Version 9 I started to play with pycarets preprocessing pipeline to see if I can improve the score. Additionally I included a voting regressor that averages the predictions of 3 models

In [ ]:
pip install pycaret --quiet

In [ ]:
# import packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import pycaret
from pycaret.regression import *
print('PyCaret: %s' % pycaret.__version__)

import matplotlib
import matplotlib.pyplot as plt
print('Matplotlib: {}'.format(matplotlib.__version__))

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# read in competition data
df_train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')

I checked quickly the data types and that there are no missing values. Nothing more. Now let's throw the data into pycaret.

In [ ]:
# create experiment to setup pycaret environment and define pre-processing pipeline
# set silent=True only after you checked the data types! this removes the user prompt and allows to run "save & run all", i.e store the notebook output
pc_experiment = setup(data=df_train, target=df_train.columns[-1], 
                      normalize = True, transformation = True, 
                      silent= True, log_experiment=True, experiment_name="TPS8_normalize")

The data has been split into Transformed Train Set and Transformed Test Set. The split was done with the default of 70/30. The Transformed Test Set (30%) is the validation set (holdout set) that is used later to assess model performance. It's important to note that the validation set is not used for cross validation. Cross validation is done on the Transformed Training Set (70%) ([see here](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)). Pycaret will use 10-fold cross validation.

We can see here that Pycaret did not create any additional features (like it would e.g. do when one hot encoding categorical features): The Transformed Train Set and the Transformed Test Set have still 100 features. 

In [ ]:
# let's see which models are available for regression in Pycaret
#models()

In [ ]:
# let's have a look at the normalized and transformed data
get_config('X_train')

In [ ]:
get_config('y_train')

In [ ]:
# compare different baseline models with 5 fold cross-validation, I ran this only once due to the execution time
#best_3_models = compare_models(fold=5, n_select=3)


# RSME with defaults
# catboost   7.8681
# lightgbm   7.8729
# gbr        7.8866
# ... lr     7.9386

# RSME with normalize=True and transform=True
# catboost   7.8569
# lightgbm   7.8619
# gbr        7.8746
# lr         7.8914

**Oberservations while waiting for pycaret to finish model evaluation:**

The first three quaters on the processing bar passed quite quickly. Then the visible progress stalled. "Estimator" is changing from time to time... so it seems to be still running.

Pycaret runs algorithms I haven't even heard of...

I have time to work out, clean and read a book while waiting.

....................................................................................Now, it's done and the winners are: **CatBoost, LightGBM and GBR**!

In [ ]:
# create model - this function uses the default hyperparameters
# you actually don't need this line if you run everything in one session. best_model would already return the trained best model
# see here: https://towardsdatascience.com/5-things-you-are-doing-wrong-in-pycaret-e01981575d2a
catboost = create_model('catboost')
lightgbm =  create_model('lightgbm')
lr = create_model('lr') # use linear regression (4th place) instead of gbr (3rd place), because gbr has huge training time

In [ ]:
# have a look at the used hyperparameters
print(lightgbm)
print(lr)
print(catboost) # for catboost this does not give the hyperparameteres, use the line below instead
catboost.get_all_params()

In [ ]:
# compare the predictions (yhat) to the ground truth (y)
modellist = [catboost, lightgbm, lr]
# fig, ax = plt.subplots(1, 3, figsize=(20,10)) # I could not figure out how to get the three plots in one row, plot_model does not support using ax=i

plot_model(catboost, plot = 'error')

In [ ]:
plot_model(lightgbm, plot = 'error')

In [ ]:
plot_model(lr, plot = 'error')

The models are getting more and more conservative, none of them is able to predict higher values.

In [ ]:
# build voting regressor that uses the top 3 models and averages the individual predictions
# the resulting blended model can be used like any other model in pycaret
blender_top3 = blend_models(estimator_list = modellist)

In [ ]:
# prediction on validation/holdout set to assess model performance
holdout_predictions = predict_model(blender_top3)
holdout_predictions.head()

I am not really satisfied with the RSME of the blended model. It is worse than the RSME of catboost on the untransformed dataset (which was 7.8658). So let's see what catboost alone gets with the transformed dataset. (-> 7.8767). This is basically the same as with blending. So, at least on the holdout set, blending seems to have no big effect.

In [ ]:
# prediction on validation/holdout set to assess model performance
holdout_predictions_cb = predict_model(catboost)
holdout_predictions_cb.head()

In [ ]:
# finalize model: re-train on whole training data (=Transformed Train Set (70%)+ Transformed Test Set (30%))
final_model = finalize_model(blender_top3)

In [ ]:
# predict for test set
test_predictions = predict_model(final_model, data=df_test)
test_predictions.head()

In [ ]:
# check which format the submission needs to have
sample_submission.head()

In [ ]:
# prepare submission file
submission = test_predictions[['id', 'Label']]
submission = submission.rename(columns={"Label": "loss"})
submission.to_csv('submission_blender.csv', index=False)
submission.head()

**Conclusions**:
- I learned a lot about pycaret
- I feel strange, because I still don't know much about the data I used
- The tuning process in Pycaret did not give the results I hoped for. I did not understand yet why.
- I am impressed about the results: Looking at the leaderboard the public score is well in line with what the other competitors get. And I did not spend any time on the data itself, just on learning to use pycaret

**What are your experiences with using Auto ML tools? I'd be happy if you comment below.**